In [73]:
# instalar as dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install fsspec
!pip install gcsfs

In [180]:
#BIBLIOTECAS
!pip install pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType, LongType
from pyspark.sql.window import Window
import pandas as pd
from pyspark.sql.functions import monotonically_increasing_id 

In [75]:
import os
from google.cloud import storage

In [76]:
#CONFIGURAR A SPARKSESSION

spark = (SparkSession.builder\
        .master("local")\
        .appName("ProjetoFinal_Pyspark")\
        .config("spark.ui.port", "4050")\
        .getOrCreate())

spark

In [175]:
# Abrindo o arquivo
serviceaccount = '/content/Chave_Ingestao_Apache.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = serviceaccount

dfspark = pd.read_csv('gs://projeto-final-grupo09/saida_dados/PJFINALzapLimpo.csv')

# CSV 
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","


customSchema = StructType([
  StructField("nome_empresa", StringType(), True),
  StructField("tipo_imovel", StringType(), True),
  StructField("cidade_imovel", StringType(), True),
  StructField("site_pesquisa", StringType(), True),
  StructField("estado_imovel", StringType(), True),
  StructField("qtde_banheiros", IntegerType(), True),
  StructField("qdte_quartos", IntegerType(), True),
  StructField("tipo_contrato", StringType(), True),
  StructField("mobilhado", StringType(), True),
  StructField("academia", StringType(), True),
  StructField("desocupado", StringType(), True),
  StructField("garagem", IntegerType(), True),
  StructField("area_lazer", StringType(), True),
  StructField("piscina", StringType(), True),
  StructField("plataforma_digital", StringType(), True),
  StructField("preco_condominio", DoubleType(), True),
  StructField("preco_periodo", StringType(), True),
  StructField("valoraluguel", DoubleType(), True),
  StructField("valor_venda", DoubleType(), True),
  StructField("iptu_anual", DoubleType(), True),
  StructField("qtde_suites", IntegerType(), True),
  StructField("quadra_tenis", StringType(), True),
  StructField("area_total_m2", DoubleType(), True),
  StructField("andar", IntegerType(), True),
  StructField("comercial_residencial", StringType(), True),
  StructField("data", DateType(), True)
])

df1 = spark.read.load(file_location, format="csv", header="true", sep=',', schema=customSchema)

df1.show(5)
df1.printSchema() 

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


+--------------------+------------+--------------+-------------+--------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+------------+-----------+----------+-----------+------------+-------------+-----+---------------------+----------+
|        nome_empresa| tipo_imovel| cidade_imovel|site_pesquisa| estado_imovel|qtde_banheiros|qdte_quartos|tipo_contrato|mobilhado|academia|desocupado|garagem|area_lazer|piscina|plataforma_digital|preco_condominio|preco_periodo|valoraluguel|valor_venda|iptu_anual|qtde_suites|quadra_tenis|area_total_m2|andar|comercial_residencial|      data|
+--------------------+------------+--------------+-------------+--------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+------------+-----------+----------+-----------+------------+-------------+-----+-----

In [179]:
df_index = df1.select("*").withColumn("id", monotonically_increasing_id())
df_index.show()

+--------------------+------------+--------------------+-------------+-----------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+------------+-----------+----------+-----------+------------+-------------+-----+---------------------+----------+---+
|        nome_empresa| tipo_imovel|       cidade_imovel|site_pesquisa|    estado_imovel|qtde_banheiros|qdte_quartos|tipo_contrato|mobilhado|academia|desocupado|garagem|area_lazer|piscina|plataforma_digital|preco_condominio|preco_periodo|valoraluguel|valor_venda|iptu_anual|qtde_suites|quadra_tenis|area_total_m2|andar|comercial_residencial|      data| id|
+--------------------+------------+--------------------+-------------+-----------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+------------+-----------+----------+-----------+---

In [173]:
# Alterando nomes das colunas "valoraluguel", "valor_venda" e "qdte_quartos"
df1 = df1.withColumnRenamed("valoraluguel", "valor_aluguel")
df1 = df1.withColumnRenamed("valor_venda", "valor_venda_mil")
df1 = df1.withColumnRenamed("qdte_quartos", "qtde_quartos")
df1.show(1)

+--------------------+------------+-------------+-------------+-------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+-------------+---------------+----------+-----------+------------+-------------+-----+---------------------+----------+-----+
|        nome_empresa| tipo_imovel|cidade_imovel|site_pesquisa|estado_imovel|qtde_banheiros|qtde_quartos|tipo_contrato|mobilhado|academia|desocupado|garagem|area_lazer|piscina|plataforma_digital|preco_condominio|preco_periodo|valor_aluguel|valor_venda_mil|iptu_anual|qtde_suites|quadra_tenis|area_total_m2|andar|comercial_residencial|      data|index|
+--------------------+------------+-------------+-------------+-------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+-------------+---------------+----------+-----------+------------+----

In [79]:
#Drop coluna "descocupado"
df1.drop('desocupado')
df1.show(5)

+--------------------+------------+--------------+-------------+--------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+-------------+---------------+----------+-----------+------------+-------------+-----+---------------------+----------+
|        nome_empresa| tipo_imovel| cidade_imovel|site_pesquisa| estado_imovel|qtde_banheiros|qtde_quartos|tipo_contrato|mobilhado|academia|desocupado|garagem|area_lazer|piscina|plataforma_digital|preco_condominio|preco_periodo|valor_aluguel|valor_venda_mil|iptu_anual|qtde_suites|quadra_tenis|area_total_m2|andar|comercial_residencial|      data|
+--------------------+------------+--------------+-------------+--------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+-------------+---------------+----------+-----------+------------+----------

In [80]:
#Quantidade de cada tipo de imóvel presente no dataframe em ordem decrescente
df1.groupBy(F.col("tipo_imovel")).count().sort("count", ascending=False).show()

+-------------------+-----+
|        tipo_imovel|count|
+-------------------+-----+
|       apartamentos| 7440|
|              casas| 6768|
|casas-de-condominio| 6720|
|               flat| 5208|
|          cobertura| 4763|
|         quitinetes| 4392|
|      casas-de-vila|  354|
|             studio|  127|
+-------------------+-----+



In [107]:
# Descrição do imovel por tipo e cômodos
df1.select(F.col("tipo_imovel"),
           F.col("qtde_quartos"), 
           F.col("qtde_suites"),
           F.col("qtde_banheiros"),  
           F.col("quadra_tenis"), 
           F.col("piscina"), 
           F.col("area_lazer"), 
           F.col("garagem")).show(10)

+------------+------------+-----------+--------------+------------+-------+----------+-------+
| tipo_imovel|qtde_quartos|qtde_suites|qtde_banheiros|quadra_tenis|piscina|area_lazer|garagem|
+------------+------------+-----------+--------------+------------+-------+----------+-------+
|apartamentos|           2|          0|             1|         Não|    Não|       Não|      0|
|apartamentos|           3|          1|             3|         Não|    Não|       Não|      1|
|apartamentos|           3|          1|             3|         Não|    Não|       Não|      2|
|apartamentos|           1|          0|             2|         Não|    Sim|       Não|      1|
|apartamentos|           2|          2|             3|         Não|    Sim|       Sim|      2|
|apartamentos|           2|          1|             1|         Não|    Sim|       Não|      1|
|apartamentos|           1|          1|             2|         Não|    Sim|       Não|      1|
|apartamentos|           2|          1|           

In [81]:
#Quantidade de imóveis anunciados, em ordem decrescente, em cidades do estado de São Paulo
df1.groupBy(F.col("cidade_imovel")).count().sort("count", ascending=False).show()

+--------------------+-----+
|       cidade_imovel|count|
+--------------------+-----+
|           São Paulo|12533|
|      Rio de Janeiro| 3639|
|            Campinas| 2327|
|        Porto Alegre| 1473|
|      Belo Horizonte|  873|
|            Curitiba|  798|
|      Ribeirão Preto|  716|
|             Barueri|  634|
| Santana de Parnaíba|  546|
|       Florianópolis|  399|
|             Niterói|  381|
|         Santo André|  375|
| São José dos Campos|  345|
|São Bernardo do C...|  316|
|            Valinhos|  302|
|            Sorocaba|  300|
|             Jundiaí|  296|
|           Fortaleza|  296|
|            Londrina|  280|
|             Vinhedo|  260|
+--------------------+-----+
only showing top 20 rows



In [112]:
#Exibindo "studios" no estado de São Paulo
df1.where((F.col("tipo_imovel") == "studio") & (F.col("estado_imovel") == "São Paulo")).show()

+--------------------+-----------+--------------+-------------+-------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+-------------+---------------+----------+-----------+------------+-------------+-----+---------------------+----------+
|        nome_empresa|tipo_imovel| cidade_imovel|site_pesquisa|estado_imovel|qtde_banheiros|qtde_quartos|tipo_contrato|mobilhado|academia|desocupado|garagem|area_lazer|piscina|plataforma_digital|preco_condominio|preco_periodo|valor_aluguel|valor_venda_mil|iptu_anual|qtde_suites|quadra_tenis|area_total_m2|andar|comercial_residencial|      data|
+--------------------+-----------+--------------+-------------+-------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+-------------+---------------+----------+-----------+------------+-------------+--

In [116]:
#Imóveis mobilhados no estado de Minas Gerais com 2 ou mais quartos

filtroMG = F.col("estado_imovel") == "Minas Gerais"
filtroMobS = F.col("mobilhado") == "Sim"
filtro2q = F.col("qtde_quartos") >= 2

df1.select(F.col("tipo_imovel"),
          F.col("qtde_quartos"),
          F.col("cidade_imovel"),
          F.col("mobilhado")).filter(filtroMG & filtroMobS & filtro2q).show()

+-------------------+------------+--------------+---------+
|        tipo_imovel|qtde_quartos| cidade_imovel|mobilhado|
+-------------------+------------+--------------+---------+
|       apartamentos|           2|     Nova Lima|      Sim|
|       apartamentos|           3|Belo Horizonte|      Sim|
|       apartamentos|           2|Belo Horizonte|      Sim|
|       apartamentos|           5|        Viçosa|      Sim|
|       apartamentos|           2|Belo Horizonte|      Sim|
|       apartamentos|           3|Belo Horizonte|      Sim|
|       apartamentos|           3|Belo Horizonte|      Sim|
|       apartamentos|           2|Belo Horizonte|      Sim|
|       apartamentos|           3|     Nova Lima|      Sim|
|       apartamentos|           3|Belo Horizonte|      Sim|
|       apartamentos|           3|     Nova Lima|      Sim|
|       apartamentos|           3|Belo Horizonte|      Sim|
|       apartamentos|           3|       Uberaba|      Sim|
|       apartamentos|           3|Belo H

In [84]:
#Quantidade de imóveis "encontrados" por cada site de pesquisa
df1.groupBy(F.col("site_pesquisa")).count().sort("count", ascending=False).show()

+-------------------+-----+
|      site_pesquisa|count|
+-------------------+-----+
|             GOOGLE|35252|
|           GRUPOZAP|  333|
|      nao informado|  138|
|          PUBLISHER|   32|
|GeoPointSource_NONE|   17|
+-------------------+-----+



In [85]:
#Valor máximo de aluguel em cada tipo de imóvel
(df1.groupby(F.col("tipo_imovel")).agg(
    F.max("valor_aluguel").alias("valor_max_aluguel"),
    F.max("valor_venda_mil").alias("Valor_max_venda_mil")).show())

+-------------------+-----------------+-------------------+
|        tipo_imovel|valor_max_aluguel|Valor_max_venda_mil|
+-------------------+-----------------+-------------------+
|         quitinetes|            999.0|              900.0|
|       apartamentos|            997.0|              990.0|
|              casas|            980.0|              995.0|
|casas-de-condominio|            990.0|              999.0|
|             studio|            987.0|              640.0|
|               flat|            950.0|              996.4|
|      casas-de-vila|            990.0|              860.0|
|          cobertura|            996.0|              999.9|
+-------------------+-----------------+-------------------+



In [145]:
#Quantidade media de quartos por tipo de imovel
(df1.groupBy(F.col("tipo_imovel")).agg(F.round(F.mean("qtde_quartos"), 2).alias("media_qtde_quartos"))).show()

+-------------------+------------------+
|        tipo_imovel|media_qtde_quartos|
+-------------------+------------------+
|         quitinetes|               0.9|
|       apartamentos|              2.11|
|              casas|              2.75|
|casas-de-condominio|              3.45|
|             studio|              0.92|
|               flat|              1.12|
|      casas-de-vila|              2.24|
|          cobertura|              3.21|
+-------------------+------------------+



In [154]:
#Filtro por cidades iniciadaas com J
df1.filter("cidade_imovel like 'J%' ").show()

+--------------------+------------+-------------+-------------+--------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+-------------+---------------+----------+-----------+------------+-------------+-----+---------------------+----------+
|        nome_empresa| tipo_imovel|cidade_imovel|site_pesquisa| estado_imovel|qtde_banheiros|qtde_quartos|tipo_contrato|mobilhado|academia|desocupado|garagem|area_lazer|piscina|plataforma_digital|preco_condominio|preco_periodo|valor_aluguel|valor_venda_mil|iptu_anual|qtde_suites|quadra_tenis|area_total_m2|andar|comercial_residencial|      data|
+--------------------+------------+-------------+-------------+--------------+--------------+------------+-------------+---------+--------+----------+-------+----------+-------+------------------+----------------+-------------+-------------+---------------+----------+-----------+------------+-------------

In [165]:
w0 = Window.partitionBy(F.col("nome_empresa")).orderBy("preco_condominio")
(df1.withColumn("row_number", F.row_number().over(w0)).select(F.col("nome_empresa"),
                                                              F.col("tipo_imovel"),
                                                              F.col("cidade_imovel"), 
                                                              F.col("comercial_residencial"),
                                                              F.col("preco_condominio"))).show()

+--------------------+------------+--------------------+---------------------+----------------+
|        nome_empresa| tipo_imovel|       cidade_imovel|comercial_residencial|preco_condominio|
+--------------------+------------+--------------------+---------------------+----------------+
|ADI Assessoria e ...|apartamentos|           São Paulo|          RESIDENTIAL|            50.0|
|   San Remos Imóveis|apartamentos|       Florianópolis|          RESIDENTIAL|           1.961|
|Família Bacellar ...|apartamentos|      Rio de Janeiro|          RESIDENTIAL|           835.0|
|Next Soluções Imo...|apartamentos|            Campinas|          RESIDENTIAL|         134.437|
|QUINTO ANDAR SERV...|apartamentos|           São Paulo|          RESIDENTIAL|           536.0|
|FLAVIA DANIELLE P...|apartamentos|      Rio de Janeiro|          RESIDENTIAL|           809.0|
|QUINTO ANDAR SERV...|apartamentos|           São Paulo|          RESIDENTIAL|           880.0|
|LELLO IMÓVEIS TAT...|apartamentos|     

In [ ]:
#client = storage.Client()
#bucket = client.get_bucket('projeto-final-grupo09')
     
#bucket.blob('saida_dados/PJFINALzapLimpo_pyspark.csv').upload_from_string(df1.to_csv(index=False), 'text/csv')